In [1]:
#AI packages

from PIL import Image
from keras.preprocessing import image
from keras.models import load_model
from keras.layers import Dense
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from python_speech_features import mfcc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pylab
import librosa
import numpy as np
import os
from matplotlib import cm
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


In [2]:
path = os.getcwd() #current working directory

rawdf = pd.read_excel(path+'/1selcleakdata.xlsx') #load csv file 

print(len(rawdf)) #length of rawdf 

5371


In [3]:

audio_data = list() #empty list 
for i in tqdm(range(rawdf.shape[0])):
    audio_data.append(librosa.load(os.path.join(path,rawdf['fname'].iloc[i]))) #signal data + samplerate(22050) using librosa package
audio_data = np.array(audio_data)

print(audio_data)

100%|██████████| 5371/5371 [00:09<00:00, 546.71it/s]

[[array([ 0.02032471, -0.00753784, -0.0178833 , ..., -0.06256104,
       -0.05389404, -0.01464844], dtype=float32)
  22050]
 [array([ 0.01727295,  0.01202393,  0.01269531, ...,  0.00198364,
       -0.00439453, -0.01126099], dtype=float32)
  22050]
 [array([ 0.01629639,  0.01965332,  0.01803589, ...,  0.00021362,
       -0.00012207,  0.00045776], dtype=float32)
  22050]
 ...
 [array([ 8.5449219e-04, -6.1035156e-05, -2.4414062e-04, ...,
       -1.8310547e-04, -4.2724609e-04, -4.2724609e-04], dtype=float32)
  22050]
 [array([ 0.00036621,  0.00082397,  0.00073242, ...,  0.00140381,
        0.0010376 , -0.00030518], dtype=float32)
  22050]
 [array([ 0.00286865,  0.00375366,  0.00357056, ..., -0.00036621,
       -0.00036621,  0.00064087], dtype=float32)
  22050]]



<ipython-input-3-df2891a8d44d>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  audio_data = np.array(audio_data)


In [4]:
df = rawdf.copy() #copy rawdata

df['audio_waves'] = audio_data[:,0] #signal data
df['samplerate'] = audio_data[:,1] #samplerate data 

df.head()

,fname,label,audio_waves,samplerate
0,38924_20210601_09_57_35_128_L.wav,L,"[0.020324707, -0.007537842, -0.0178833, -0.024...",22050
1,38925_20210601_10_00_35_128_L.wav,L,"[0.01727295, 0.012023926, 0.0126953125, 0.0130...",22050
2,38926_20210601_10_01_57_128_L.wav,L,"[0.016296387, 0.01965332, 0.018035889, 0.01998...",22050
3,38932_20210601_10_15_32_128_L.wav,L,"[0.075927734, -0.040130615, -0.124420166, -0.0...",22050
4,38933_20210601_10_20_51_128_undefined.wav,undefined,"[0.015197754, 0.01574707, 0.015045166, 0.01467...",22050


In [5]:
df2 = df.copy() 

bit_lengths = list() #empty list for no of bits 
for i in range(df2.shape[0]):
    bit_lengths.append(len(df2['audio_waves'].iloc[i]))  #bit_lengths 
bit_lengths = np.array(bit_lengths)


#regularization 

for i in range(len(bit_lengths)):
    if (bit_lengths[i]  >= 110250) & (bit_lengths[i] < 132300): #5 seconds
        bit_lengths[i] =  110250 
        
    elif (bit_lengths[i]   >= 88200) & (bit_lengths[i]  < 110250): #4 seconds
         bit_lengths[i] = 88200    
    
    elif (bit_lengths[i]  >= 66150) & (bit_lengths[i]  < 88200): #3 seconds
        bit_lengths[i] = 66150

    elif (bit_lengths[i]  >= 44100) & (bit_lengths[i]  < 66150): #2 seconds
        bit_lengths[i] = 44100
    
    else:
        bit_lengths[i] = bit_lengths[i]
        
df2['bit_lengths'] = bit_lengths 

df2['second_lengths'] = df2['bit_lengths']/df2['samplerate']  #no of bits / samplerate = audio length(s)
df2





,fname,label,audio_waves,samplerate,bit_lengths,second_lengths
0,38924_20210601_09_57_35_128_L.wav,L,"[0.020324707, -0.007537842, -0.0178833, -0.024...",22050,110250,5.0
1,38925_20210601_10_00_35_128_L.wav,L,"[0.01727295, 0.012023926, 0.0126953125, 0.0130...",22050,110250,5.0
2,38926_20210601_10_01_57_128_L.wav,L,"[0.016296387, 0.01965332, 0.018035889, 0.01998...",22050,110250,5.0
3,38932_20210601_10_15_32_128_L.wav,L,"[0.075927734, -0.040130615, -0.124420166, -0.0...",22050,110250,5.0
4,38933_20210601_10_20_51_128_undefined.wav,undefined,"[0.015197754, 0.01574707, 0.015045166, 0.01467...",22050,110250,5.0
...,...,...,...,...,...,...
5366,82593_20211016_14_56_55_128_L.wav,L,"[0.03491211, 0.041229248, 0.04663086, 0.054412...",22050,66150,3.0
5367,82594_20211016_14_57_38_128_L.wav,L,"[-0.1244812, -0.11431885, -0.10385132, -0.0925...",22050,66150,3.0
5368,82595_20211016_14_58_35_128_L.wav,L,"[0.0008544922, -6.1035156e-05, -0.00024414062,...",22050,66150,3.0
5369,82596_20211016_15_02_55_128_undefined.wav,undefined,"[0.00036621094, 0.0008239746, 0.0007324219, -0...",22050,66150,3.0


In [7]:
df3 = df2.copy()

df3 = df3[df3['second_lengths'] >=3] #use at least over 3 seconds audio data

df3 = df3.reset_index(drop=True)

df3

df3.to_excel("zzxcvcxv.xlsx")

In [ ]:
mfcc_labels = list() #empty list for audio data label: L = leak, U = undefined(no leak)


for i in (range(len(df3['audio_waves']))):
        mfcc_labels.append(df3['label'][i])


mfcc_labels = pd.DataFrame(mfcc_labels)

print(mfcc_labels[0][77])

In [ ]:
#####draw mel-spectrogram #######################

mfcc_features = list() #empty list for mfcc data

for i in tqdm(range(len(df3['audio_waves']))):
    mfcc_features.append(mfcc(df3['audio_waves'][i]))
    

    #draw mel-spectrogram
    M = librosa.feature.melspectrogram(df3['audio_waves'][i], sr = 22050, fmax = 2800, #max hz
                                            n_fft=2048, #음성의 길이를 얼마만큼 자를 것인지
                                            hop_length=512, #얼마만큼 시간주기를 이동하면서 분석할것인지 
                                            n_mels = 96, #칼라맵의 해상도 
                                            power = 2) #default value 

    log_power = librosa.power_to_db(M, ref=np.max)  #log scale 
            
    # plotting the leak/noleak spectrogram and save as png file (just the image)
    pylab.figure(figsize=(3,3))
    pylab.axis('off') #no x or y axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    librosa.display.specshow(log_power, cmap=cm.jet)
    pylab.savefig(str(df3['label'][i]) + df3['fname'][i] + str(i)  +'.png', bbox_inches=None, pad_inches=0)
    #pylab.close()

In [ ]:
#check 

filename = 'C:/Users/user/Desktop/210905_CNN(50~2800)/code/audiodata/38971_20210601_12_56_47_129_undefined.wav' #local dir
audio = librosa.load(filename)


M = librosa.feature.melspectrogram(audio[0], sr = 22050, fmax = 22050/2, 
                                            n_fft=2048, 
                                            hop_length=512, 
                                            n_mels = 96, 
                                            power = 2) 


# power to db (log로)
log_power = librosa.power_to_db(M, ref=np.max) 

pylab.figure(figsize=(3,3))
pylab.axis('off') 
pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
librosa.display.specshow(log_power, cmap=cm.jet)

#plt.savefig(str(filename[0:5])+'.png')
plt.show()



